<h3> To fix this start from this case study provided in research paper 1 of IFAHP , implement multiplicative consistency as per the paper 1 and i got the values a little bit off </h3>
<br>the output should look like this </br> 

### Perfect Multiplicative Consistent IFPR Matrix:

| Row/Col | 1        | 2        | 3        | 4        | 5        |
|---------|----------|----------|----------|----------|----------|
| **1**   | (0.5, 0.5) | (0.6, 0.2) | (0.6923, 0.0588) | (0.7146, 0.0673) | (0.8069, 0.0204) |
| **2**   | (0.2, 0.6) | (0.5, 0.5) | (0.6, 0.2) | (0.6923, 0.0588) | (0.7146, 0.0673) |
| **3**   | (0.0588, 0.6923) | (0.2, 0.6) | (0.5, 0.5) | (0.6, 0.2) | (0.6923, 0.0588) |
| **4**   | (0.0673, 0.7146) | (0.0588, 0.6923) | (0.2, 0.6) | (0.5, 0.5) | (0.6, 0.2) |
| **5**   | (0.0204, 0.8069) | (0.0673, 0.7146) | (0.0588, 0.6923) | (0.2, 0.6) | (0.5, 0.5) |
.5        0.5       ]]] </h3>

In [109]:
import numpy as np

# Initial IFPR matrix for criteria with respect to the overall objective
R = np.array([
    [[0.5, 0.5], [0.6, 0.2], [0.6, 0.2], [0.65, 0.25], [0.65, 0.25]],
    [[0.2, 0.6], [0.5, 0.5], [0.6, 0.2], [0.65, 0.25], [0.65, 0.25]],
    [[0.4, 0.2], [0.4, 0.2], [0.5, 0.5], [0.6, 0.2], [0.6, 0.2]],
    [[0.35, 0.65], [0.35, 0.65], [0.4, 0.2], [0.5, 0.5], [0.6, 0.2]],
    [[0.35, 0.65], [0.35, 0.65], [0.4, 0.2], [0.4, 0.2], [0.5, 0.5]]
])

In [95]:
R = np.array([
    [[0.5, 0.5], [0.2, 0.6], [0.3, 0.4], [0.6, 0.2]],
    [[0.6, 0.2], [0.5, 0.5], [0.5, 0.4], [0.6, 0.4]],
    [[0.4, 0.3], [0.4, 0.5], [0.5, 0.5], [0.3, 0.2]],
    [[0.2, 0.6], [0.4, 0.6], [0.2, 0.3], [0.5, 0.5]]
])

In [29]:
R = np.array([
    [[0.5, 0.3], [0.4, 0.5], [0.2, 0.7]],
    [[0.5, 0.4], [0.5, 0.3], [0.3, 0.6]],
    [[0.7, 0.2], [0.6, 0.3], [0.5, 0.3]]
])

In [163]:
R= np.array([
[[0.5, 0.3], [0.2, 0.6], [0.1, 0.8]],
[[0.6, 0.2], [0.5, 0.3], [0.3, 0.6]], # ACTUAL IFPR MATRIX
[[0.8, 0.1], [0.6, 0.3], [0.5, 0.3]]
])

<h3> Method 1 incorporating the formula from research paper 1 of IFAHP </h3>

In [165]:
def construct_perfect_multiplicative_consistent_ifpr(R):
    """
    Constructs the Perfect Multiplicative Consistent IFPR (R_bar) from the original IFPR R.
    
    For i < k, if we define the chain over t = i+1, ..., k-1, then:
    
       μ̃(i,k) = { [∏ₜ ( μ(i,t) * μ(t,k) )]^(exponent) } /
                  { [∏ₜ ( μ(i,t) * μ(t,k) )]^(exponent) + [∏ₜ ( (1-μ(i,t)) * (1-μ(t,k)) )]^(exponent) }
    
       ν̃(i,k) is defined similarly using ν.
    
    The exponent is chosen as follows:
      - If chain length = (k - i - 1) equals 1, exponent = 1.
      - If chain length > 1, exponent = 0.5 (i.e. take square root).
    
    For adjacent indices (k = i+1), the original values are kept.
    The lower-triangular part is filled by reciprocity:
      R_bar[k,i] = (ν̃(i,k), μ̃(i,k))
    """
    n = R.shape[0]
    R_bar = np.zeros((n, n, 2), dtype=float)

    # 1. Set diagonal entries to (0.5, 0.5)
    for i in range(n):
        R_bar[i, i] = [0.5, 0.5]

    # 2. Compute upper-triangular entries
    for i in range(n):
        for k in range(i+1, n):
            if k == i + 1:
                # For adjacent indices, simply use the original value.
                R_bar[i, k] = R[i, k]
            else:
                exponent = 1.0 / (k - i - 1)
                # Compute chain-products for membership
                prod_mu = 1.0
                prod_1mu = 1.0
                for t in range(i+1, k):
                    prod_mu   *= (R[i, t, 0] * R[t, k, 0])
                    prod_1mu  *= ((1 - R[i, t, 0]) * (1 - R[t, k, 0]))
                left_mu  = prod_mu ** exponent
                right_mu = prod_1mu ** exponent
                denom_mu = left_mu + right_mu
                new_mu = left_mu / denom_mu if denom_mu != 0 else 0.5

                # Compute chain-products for non-membership
                prod_nu = 1.0
                prod_1nu = 1.0
                for t in range(i+1, k):
                    prod_nu   *= (R[i, t, 1] * R[t, k, 1])
                    prod_1nu  *= ((1 - R[i, t, 1]) * (1 - R[t, k, 1]))
                left_nu  = prod_nu ** exponent
                right_nu = prod_1nu ** exponent
                denom_nu = left_nu + right_nu
                new_nu = left_nu / denom_nu if denom_nu != 0 else 0.5

                R_bar[i, k] = [new_mu, new_nu]

    # 3. Fill the lower-triangular part using reciprocity:
    # If R_bar[i,k] = (μ, ν), then set R_bar[k,i] = (ν, μ).
    for i in range(n):
        for k in range(i+1, n):
            mu_val, nu_val = R_bar[i, k]
            R_bar[k, i] = [nu_val, mu_val]

    return R_bar

In [167]:
# Construct the Perfect Multiplicative Consistent IFPR
R_bar = construct_perfect_multiplicative_consistent_ifpr(R)

# Print results with better formatting
np.set_printoptions(precision=4, suppress=True)
print("Original R (upper triangle):")
print(R[..., 0], "\n", R[..., 1], "\n")  # Print membership (μ) and non-membership (ν) matrices

print("Perfect Multiplicative Consistent IFPR (R_bar):")

# Dynamically determine the size of the matrix
n = R.shape[0]  # Number of rows (and columns, since it's an N x N matrix)

# Loop through each row and column dynamically
for i in range(n):
    row_str = []
    for k in range(n):
        mu_val = R_bar[i, k, 0]
        nu_val = R_bar[i, k, 1]
        row_str.append(f"({mu_val:.4f}, {nu_val:.4f})")
    print(f"Row {i+1}:", "\t".join(row_str))

Original R (upper triangle):
[[0.5 0.2 0.1]
 [0.6 0.5 0.3]
 [0.8 0.6 0.5]] 
 [[0.3 0.6 0.8]
 [0.2 0.3 0.6]
 [0.1 0.3 0.3]] 

Perfect Multiplicative Consistent IFPR (R_bar):
Row 1: (0.5000, 0.5000)	(0.2000, 0.6000)	(0.0968, 0.6923)
Row 2: (0.6000, 0.2000)	(0.5000, 0.5000)	(0.3000, 0.6000)
Row 3: (0.6923, 0.0968)	(0.6000, 0.3000)	(0.5000, 0.5000)


In [169]:
def check_consistency(R, R_bar, tau=0.1):
    """
    Computes the distance d between the original IFPR matrix R and 
    the perfectly multiplicative consistent IFPR matrix R_bar using:
    
      d(R, R_bar) = (1 / ((n-1) * (n-2))) * sum_{i=1}^{n} sum_{k=i+1}^{n} 
                     [|mu_bar(i,k) - mu(i,k)| + |nu_bar(i,k) - nu(i,k)| + |pi_bar(i,k) - pi(i,k)|],
                     
    where for each entry,
      pi(i,k) = 1 - mu(i,k) - nu(i,k) and
      pi_bar(i,k) = 1 - mu_bar(i,k) - nu_bar(i,k).
    
    Parameters:
      R      : Original IFPR matrix (n x n x 2 numpy array)
      R_bar  : Consistent IFPR matrix (n x n x 2 numpy array)
      tau    : Threshold for acceptable consistency (default: 0.1)
    
    Returns:
      (bool) True if d < tau, else False.
    """
    n = R.shape[0]
    total_diff = 0.0
    
    # Loop over the upper triangular indices (i < k)
    for i in range(n):
        for k in range(i+1, n):
            mu = R[i, k, 0]
            nu = R[i, k, 1]
            pi = 1 - mu - nu
            
            mu_bar = R_bar[i, k, 0]
            nu_bar = R_bar[i, k, 1]
            pi_bar = 1 - mu_bar - nu_bar
            
            diff = abs(mu_bar - mu) + abs(nu_bar - nu) + abs(pi_bar - pi)
            total_diff += diff
    
    # Use normalization factor as in the paper: (n-1)*(n-2) or 2*(n-1)(n-2) idk man
    denominator = (n - 1) * (n - 2)
    d = total_diff / denominator
    
    print("Distance d(R, R_bar) =", d)
    return d < tau

In [171]:
R_bar = construct_perfect_multiplicative_consistent_ifpr(R)
    
# Check consistency:
consistent = check_consistency(R, R_bar, tau=0.1)
if consistent:
    print("The IFPR is consistent (d < 0.1).")
else:
    print("The IFPR is NOT consistent (d >= 0.1).")

Distance d(R, R_bar) = 0.11091811414392064
The IFPR is NOT consistent (d >= 0.1).


In [173]:
def repair_ifahp_algorithm_2(R, sigma=0.5, tau=0.1, max_iter=100):
    """
    Repairs the IFPR R using the fused intuitionistic preference algorithm (Algorithm 2)
    until the consistency measure d(R, R_bar) < tau.
    
    The update rules for off-diagonal entries are:
    
      μ^(p+1)(i,k) = [ (μ^(p)(i,k))^(1-σ) * (μ̄(i,k))^σ ] /
                      { (μ^(p)(i,k))^(1-σ) * (μ̄(i,k))^σ + (1-μ^(p)(i,k))^(1-σ) * (1-μ̄(i,k))^σ }
    
      ν^(p+1)(i,k) = [ (ν^(p)(i,k))^(1-σ) * (ν̄(i,k))^σ ] /
                      { (ν^(p)(i,k))^(1-σ) * (ν̄(i,k))^σ + (1-ν^(p)(i,k))^(1-σ) * (1-ν̄(i,k))^σ }
    
    Diagonal entries are fixed to (0.5, 0.5) and the lower-triangular part is
    updated via reciprocity.
    """
    # First, compute the perfect IFPR matrix.
    R_bar = construct_perfect_multiplicative_consistent_ifpr(R)
    print("Initial consistency check:")
    if check_consistency(R, R_bar, tau):
        print("R is already consistent with R_bar; no repair needed.")
        return R

    # Start the iterative repair process.
    R_current = R.copy()
    n = R_current.shape[0]
    
    for p in range(max_iter):
        R_next = np.copy(R_current)
        
        # Update all off-diagonal entries using the ratio-based formulas.
        for i in range(n):
            for k in range(n):
                if i != k:
                    mu_p = R_current[i, k, 0]
                    nu_p = R_current[i, k, 1]
                    mu_bar = R_bar[i, k, 0]
                    nu_bar = R_bar[i, k, 1]
                    
                    # Membership update:
                    num_mu = (mu_p ** (1 - sigma)) * (mu_bar ** sigma)
                    den_mu = num_mu + ((1 - mu_p) ** (1 - sigma)) * ((1 - mu_bar) ** sigma)
                    mu_next = num_mu / den_mu if den_mu != 0 else 0.5
                    
                    # Non-membership update:
                    num_nu = (nu_p ** (1 - sigma)) * (nu_bar ** sigma)
                    den_nu = num_nu + ((1 - nu_p) ** (1 - sigma)) * ((1 - nu_bar) ** sigma)
                    nu_next = num_nu / den_nu if den_nu != 0 else 0.5
                    
                    R_next[i, k, 0] = mu_next
                    R_next[i, k, 1] = nu_next
        
        # Enforce the diagonal to remain (0.5, 0.5)
        for i in range(n):
            R_next[i, i] = [0.5, 0.5]
        
        # Enforce reciprocity for the lower-triangular part:
        for i in range(n):
            for k in range(i+1, n):
                mu_val, nu_val = R_next[i, k]
                R_next[k, i] = [nu_val, mu_val]
        
        print(f"Iteration {p+1} consistency check:")
        if check_consistency(R_next, R_bar, tau):
            print(f"Repair successful after {p+1} iterations.")
            return R_next
        
        R_current = R_next
    
    print("Max iterations reached; returning final repaired matrix.")
    return R_current

In [175]:
R_repaired = repair_ifahp_algorithm_2(R, sigma=0.8, tau=0.1, max_iter=100)
    
# --- Print the repaired IFPR matrix ---
print("\nRepaired IFPR matrix:")
for i in range(n):
    row_str = []
    for k in range(n):
        mu_val = R_repaired[i, k, 0]
        nu_val = R_repaired[i, k, 1]
        row_str.append(f"({mu_val:.4f}, {nu_val:.4f})")
    print("Row", i+1, ":", "\t".join(row_str))

Initial consistency check:
Distance d(R, R_bar) = 0.11091811414392064
Iteration 1 consistency check:
Distance d(R, R_bar) = 0.024593633427534276
Repair successful after 1 iterations.

Repaired IFPR matrix:
Row 1 : (0.5000, 0.5000)	(0.2000, 0.6000)	(0.0974, 0.7163)
Row 2 : (0.6000, 0.2000)	(0.5000, 0.5000)	(0.3000, 0.6000)
Row 3 : (0.7163, 0.0974)	(0.6000, 0.3000)	(0.5000, 0.5000)


In [177]:
def compute_criterion_weights(R):
    """
    Computes the intuitionistic fuzzy weights ω_i = (μ_i, ν_i) for each row i
    using Formula (26).
    """
    n = R.shape[0]
    weights = np.zeros((n, 2), dtype=float)
    
    # Calculate the denominators first (they are the same for all weights)
    sum_all_mu = 0.0
    sum_all_one_minus_nu = 0.0
    
    for i in range(n):
        for k in range(n):
            sum_all_mu += R[i, k, 0]  # Sum of all μ(i,k)
            sum_all_one_minus_nu += (1.0 - R[i, k, 1])  # Sum of all (1-ν(i,k))
    
    # Now calculate weights for each criterion
    for i in range(n):
        sum_mu_i = sum(R[i, k, 0] for k in range(n))  # Sum of μ(i,k) for row i
        sum_one_minus_nu_i = sum(1.0 - R[i, k, 1] for k in range(n))  # Sum of (1-ν(i,k)) for row i
        
        # Calculate weights according to formula 26
        if sum_all_one_minus_nu <= 1e-15 or sum_all_mu <= 1e-15:
            w_mu = 0.5
            w_nu = 0.5
        else:
            w_mu = sum_mu_i / sum_all_one_minus_nu
            w_nu = 1.0 - (sum_one_minus_nu_i / sum_all_mu)
        
        weights[i, 0] = w_mu
        weights[i, 1] = w_nu
    
    return weights

In [179]:
weights = compute_criterion_weights(R_repaired)

print("\nCriterion Weights (μ, ν):")
for i, (mu_w, nu_w) in enumerate(weights, start=1):
    print(f"ω_{i} = ({mu_w:.4f}, {nu_w:.4f})")


Criterion Weights (μ, ν):
ω_1 = (0.1599, 0.7051)
ω_2 = (0.2808, 0.5764)
ω_3 = (0.3642, 0.4761)


In [181]:
def convert_to_crisp_weights(weights):
    """
    Converts intuitionistic fuzzy weights to crisp weights.
    
    Args:
    - weights: np.ndarray of shape (n, 2) where weights[i, 0] is μ_i and weights[i, 1] is ν_i
    
    Returns:
    - crisp_weights: Normalized crisp weights
    """
    n = weights.shape[0]
    crisp_values = np.zeros(n)
    
    for i in range(n):
        mu = weights[i, 0]
        nu = weights[i, 1]
        
        # Score function: S(ω_i) = μ_i - ν_i
        score = mu - nu
        
        # Accuracy function: H(ω_i) = μ_i + ν_i
        accuracy = mu + nu
        
        # Combined score (this is one approach)
        crisp_values[i] = 0.5 * (score + 1) * (1 + accuracy - score)
    
    # Normalize to ensure sum is 1
    sum_crisp = np.sum(crisp_values)
    if sum_crisp > 0:
        crisp_weights = crisp_values / sum_crisp
    else:
        # Equal weights if all values are 0
        crisp_weights = np.ones(n) / n
        
    return crisp_weights

In [183]:
crisp_weights = convert_to_crisp_weights(weights)
print(crisp_weights)

[0.2522 0.3489 0.3989]


In [185]:
import pandas as pd

# Load the dataset
data = pd.read_csv("excel_file/Requirements.csv")
data = data.drop(columns=['Pre-requisite', 'Test Cases'])
# Map Importance values ('H', 'M', 'L') to numerical values (4, 2.5, 1)
importance_mapping = {'H': 9, 'M': 5, 'L': 3}
data['Importance'] = data['Importance'].map(importance_mapping)

# Extract the decision matrix
decision_matrix = data[['Cost', 'Value', 'Importance']].values

# Print the decision matrix
print("Decision Matrix:")
print(decision_matrix)

Decision Matrix:
[[8 9 9]
 [3 2 9]
 [7 8 9]
 [4 3 5]
 [6 7 9]
 [7 8 9]
 [6 3 3]
 [5 4 3]
 [7 1 3]
 [7 3 3]
 [5 6 5]
 [6 4 5]
 [4 3 9]
 [5 7 5]
 [7 2 3]
 [7 6 5]
 [3 2 3]
 [4 5 5]
 [5 4 5]
 [5 4 5]
 [6 5 5]
 [6 6 5]
 [8 6 5]
 [7 5 5]
 [5 9 9]
 [2 6 5]
 [6 9 9]
 [7 8 9]
 [7 4 5]
 [4 8 5]
 [6 6 5]
 [8 5 3]
 [4 9 9]
 [6 8 5]
 [3 5 3]
 [4 8 9]
 [2 4 5]
 [5 8 9]
 [5 8 9]
 [7 7 5]
 [4 8 5]
 [2 8 5]
 [1 7 9]
 [5 6 5]
 [6 4 5]
 [5 9 5]
 [3 8 9]
 [6 7 9]
 [5 8 9]
 [5 7 9]
 [6 6 5]
 [4 8 9]
 [6 5 5]
 [5 3 5]
 [6 4 3]
 [2 7 5]
 [3 6 9]
 [5 7 9]
 [6 8 9]
 [6 7 5]
 [7 5 3]
 [5 7 9]
 [4 3 9]
 [3 6 9]
 [5 3 9]
 [4 3 9]
 [6 4 9]
 [3 3 9]
 [6 4 3]
 [3 1 3]
 [5 3 5]
 [4 6 9]
 [6 6 5]
 [5 7 5]
 [4 4 5]
 [3 4 5]
 [7 5 5]
 [4 3 9]
 [5 7 9]
 [3 3 3]
 [4 2 5]
 [3 2 3]
 [4 6 9]
 [5 5 9]
 [5 5 9]
 [6 8 5]
 [4 7 9]
 [5 6 9]
 [4 5 5]
 [3 7 9]
 [4 6 5]
 [6 9 9]
 [5 5 5]
 [7 8 9]
 [8 8 5]
 [6 4 9]
 [7 7 3]
 [4 8 5]
 [4 5 9]
 [5 4 9]]


In [187]:
def determine_best_and_worst(decision_matrix, criteria_types):
    """
    Determines the best and worst values for each criterion.
    Args:
    - decision_matrix: A numpy array of shape (m, n), where m is the number of alternatives and n is the number of criteria.
    - criteria_types: A list indicating the type of each criterion ('max' for beneficial, 'min' for non-beneficial).
    Returns:
    - f_star: The best values for each criterion.
    - f_minus: The worst values for each criterion.
    """
    f_star = []
    f_minus = []

    for j, criterion_type in enumerate(criteria_types):
        if criterion_type == 'max':  # Beneficial criterion
            f_star.append(np.max(decision_matrix[:, j]))
            f_minus.append(np.min(decision_matrix[:, j]))
        elif criterion_type == 'min':  # Non-beneficial criterion
            f_star.append(np.min(decision_matrix[:, j]))
            f_minus.append(np.max(decision_matrix[:, j]))

    return np.array(f_star), np.array(f_minus)

# Define criteria types
criteria_types = ['min', 'max', 'max']  # Cost: min, Value: max, Importance: max

# Determine best and worst values
f_star, f_minus = determine_best_and_worst(decision_matrix, criteria_types)
print("f_star, f_minus:" , f_star, f_minus)

f_star, f_minus: [1 9 9] [8 1 3]


In [189]:
def substitute_values(decision_matrix, weights, f_star, f_minus):
    """
    Substitutes each value in the decision matrix with the given formula.
    Args:
    - decision_matrix: The decision matrix.
    - weights: The weights for each criterion.
    - f_star: The best values for each criterion.
    - f_minus: The worst values for each criterion.
    Returns:
    - substituted_matrix: The substituted decision matrix.
    """
    m, n = decision_matrix.shape
    substituted_matrix = np.zeros_like(decision_matrix, dtype=float)

    for i in range(m):
        for j in range(n):
            numerator = f_star[j] - decision_matrix[i, j]
            denominator = f_star[j] - f_minus[j] + 1e-10
            substituted_matrix[i, j] = crisp_weights[j] * (numerator / denominator)

    return substituted_matrix

# Substitute values in the decision matrix
substituted_matrix = substitute_values(decision_matrix, crisp_weights, f_star, f_minus)
print("\nSubstituted Decision Matrix:")
print(substituted_matrix)


Substituted Decision Matrix:
[[ 0.2522  0.      0.    ]
 [ 0.0721  0.3053  0.    ]
 [ 0.2162  0.0436  0.    ]
 [ 0.1081  0.2617  0.2659]
 [ 0.1802  0.0872  0.    ]
 [ 0.2162  0.0436  0.    ]
 [ 0.1802  0.2617  0.3989]
 [ 0.1441  0.218   0.3989]
 [ 0.2162  0.3489  0.3989]
 [ 0.2162  0.2617  0.3989]
 [ 0.1441  0.1308  0.2659]
 [ 0.1802  0.218   0.2659]
 [ 0.1081  0.2617  0.    ]
 [ 0.1441  0.0872  0.2659]
 [ 0.2162  0.3053  0.3989]
 [ 0.2162  0.1308  0.2659]
 [ 0.0721  0.3053  0.3989]
 [ 0.1081  0.1744  0.2659]
 [ 0.1441  0.218   0.2659]
 [ 0.1441  0.218   0.2659]
 [ 0.1802  0.1744  0.2659]
 [ 0.1802  0.1308  0.2659]
 [ 0.2522  0.1308  0.2659]
 [ 0.2162  0.1744  0.2659]
 [ 0.1441  0.      0.    ]
 [ 0.036   0.1308  0.2659]
 [ 0.1802  0.      0.    ]
 [ 0.2162  0.0436  0.    ]
 [ 0.2162  0.218   0.2659]
 [ 0.1081  0.0436  0.2659]
 [ 0.1802  0.1308  0.2659]
 [ 0.2522  0.1744  0.3989]
 [ 0.1081  0.      0.    ]
 [ 0.1802  0.0436  0.2659]
 [ 0.0721  0.1744  0.3989]
 [ 0.1081  0.0436  0.    

In [191]:
def calculate_s(substituted_matrix):
    """
    Calculates the utility measure (S_i) for each alternative.
    Args:
    - substituted_matrix: The substituted decision matrix.
    Returns:
    - S: The utility measure for each alternative.
    """
    S = np.sum(substituted_matrix, axis=1)
    return S

# Calculate S_i
S = calculate_s(substituted_matrix)
print("\nUtility Measure (S):", S)


Utility Measure (S): [0.2522 0.3773 0.2598 0.6357 0.2674 0.2598 0.8407 0.7611 0.964  0.8768
 0.5409 0.6641 0.3697 0.4973 0.9204 0.613  0.7762 0.5485 0.6281 0.6281
 0.6205 0.5769 0.649  0.6566 0.1441 0.4328 0.1802 0.2598 0.7002 0.4176
 0.5769 0.8256 0.1081 0.4897 0.6454 0.1517 0.52   0.1877 0.1877 0.5693
 0.4176 0.3456 0.0872 0.5409 0.6641 0.4101 0.1157 0.2674 0.1877 0.2313
 0.5769 0.1517 0.6205 0.6717 0.7971 0.3892 0.2029 0.2313 0.2238 0.5333
 0.7895 0.2313 0.3697 0.2029 0.4058 0.3697 0.3982 0.3337 0.7971 0.8198
 0.6717 0.2389 0.5769 0.4973 0.5921 0.556  0.6566 0.3697 0.2313 0.7326
 0.6793 0.7762 0.2389 0.3186 0.3186 0.4897 0.1953 0.275  0.5485 0.1593
 0.5049 0.1802 0.5845 0.2598 0.5618 0.3982 0.7023 0.4176 0.2825 0.3622]


In [193]:
def calculate_r(substituted_matrix):
    """
    Calculates the regret measure (R_i) for each alternative.
    Args:
    - substituted_matrix: The substituted decision matrix.
    Returns:
    - R: The regret measure for each alternative.
    """
    R = np.max(substituted_matrix, axis=1)
    return R

# Calculate R_i
R = calculate_r(substituted_matrix)
print("\nRegret Measure (R):", R)
len(R)


Regret Measure (R): [0.2522 0.3053 0.2162 0.2659 0.1802 0.2162 0.3989 0.3989 0.3989 0.3989
 0.2659 0.2659 0.2617 0.2659 0.3989 0.2659 0.3989 0.2659 0.2659 0.2659
 0.2659 0.2659 0.2659 0.2659 0.1441 0.2659 0.1802 0.2162 0.2659 0.2659
 0.2659 0.3989 0.1081 0.2659 0.3989 0.1081 0.2659 0.1441 0.1441 0.2659
 0.2659 0.2659 0.0872 0.2659 0.2659 0.2659 0.0721 0.1802 0.1441 0.1441
 0.2659 0.1081 0.2659 0.2659 0.3989 0.2659 0.1308 0.1441 0.1802 0.2659
 0.3989 0.1441 0.2617 0.1308 0.2617 0.2617 0.218  0.2617 0.3989 0.3989
 0.2659 0.1308 0.2659 0.2659 0.2659 0.2659 0.2659 0.2617 0.1441 0.3989
 0.3053 0.3989 0.1308 0.1744 0.1744 0.2659 0.1081 0.1441 0.2659 0.0872
 0.2659 0.1802 0.2659 0.2162 0.2659 0.218  0.3989 0.2659 0.1744 0.218 ]


100

In [197]:
def calculate_q(S, R, S_star, S_minus, R_star, R_minus, v=0.5):
    """
    Calculates the compromise solution (Q_i) for each alternative.
    Args:
    - S: The utility measure for each alternative.
    - R: The regret measure for each alternative.
    - S_star, S_minus, R_star, R_minus: Best and worst values for S_i and R_i.
    - v: Strategy weight (default is 0.5).
    Returns:
    - Q: The compromise solution for each alternative.
    """
    m = len(S)
    Q = np.zeros(m)

    for i in range(m):
        s_term = (S[i] - S_star) / (S_minus - S_star + 1e-10)
        r_term = (R[i] - R_star) / (R_minus - R_star + 1e-10)
        Q[i] = v * s_term + (1 - v) * r_term

    return Q

# Calculate S*, R*, S-, R-
S_star = np.min(S)
S_minus = np.max(S)
R_star = np.min(R)
R_minus = np.max(R)

# Calculate Q_i
Q = calculate_q(S, R, S_star, S_minus, R_star, R_minus)

print("\nCompromise Solution (Q):\n", Q)


Compromise Solution (Q):
 [0.3697 0.5222 0.3189 0.6094 0.2681 0.3189 0.9297 0.8843 1.     0.9503
 0.5553 0.6256 0.4511 0.5304 0.9751 0.5964 0.8929 0.5596 0.6051 0.6051
 0.6007 0.5759 0.617  0.6213 0.1427 0.4937 0.2184 0.3189 0.6461 0.485
 0.5759 0.9211 0.067  0.5261 0.8183 0.0919 0.5434 0.1676 0.1676 0.5715
 0.485  0.4439 0.0232 0.5553 0.6256 0.4807 0.0162 0.2681 0.1676 0.1924
 0.5759 0.0919 0.6007 0.6299 0.9048 0.4688 0.1559 0.1924 0.2432 0.551
 0.9005 0.1924 0.4511 0.1559 0.4717 0.4511 0.4007 0.4306 0.9048 0.9178
 0.6299 0.1764 0.5759 0.5304 0.5845 0.564  0.6213 0.4511 0.1924 0.8681
 0.6944 0.8929 0.1764 0.2885 0.2885 0.5261 0.1168 0.2173 0.5596 0.0643
 0.5348 0.2184 0.5802 0.3189 0.5672 0.4007 0.8508 0.485  0.268  0.3801]


In [199]:
def rank_alternatives(Q, S, R):
    """
    Ranks alternatives based on Q, S, and R.
    Args:
    - Q: The compromise solution for each alternative.
    - S: The utility measure for each alternative.
    - R: The regret measure for each alternative.
    Returns:
    - rankings: A list of tuples containing the alternative index and its Q, S, and R values, sorted by Q.
    """
    rankings = sorted(enumerate(zip(Q, S, R)), key=lambda x: (x[1][0], x[1][1], x[1][2]))
    return rankings

# Rank alternatives
rankings = rank_alternatives(Q, S, R)

# Print rankings
print("\nRanked Alternatives:")
for rank, (index, (q, s, r)) in enumerate(rankings, start=1):
    print(f"Rank {rank}: Alternative {index + 1}, Q = {q:.4f}, S = {s:.4f}, R = {r:.4f}")


Ranked Alternatives:
Rank 1: Alternative 47, Q = 0.0162, S = 0.1157, R = 0.0721
Rank 2: Alternative 43, Q = 0.0232, S = 0.0872, R = 0.0872
Rank 3: Alternative 90, Q = 0.0643, S = 0.1593, R = 0.0872
Rank 4: Alternative 33, Q = 0.0670, S = 0.1081, R = 0.1081
Rank 5: Alternative 36, Q = 0.0919, S = 0.1517, R = 0.1081
Rank 6: Alternative 52, Q = 0.0919, S = 0.1517, R = 0.1081
Rank 7: Alternative 87, Q = 0.1168, S = 0.1953, R = 0.1081
Rank 8: Alternative 25, Q = 0.1427, S = 0.1441, R = 0.1441
Rank 9: Alternative 57, Q = 0.1559, S = 0.2029, R = 0.1308
Rank 10: Alternative 64, Q = 0.1559, S = 0.2029, R = 0.1308
Rank 11: Alternative 38, Q = 0.1676, S = 0.1877, R = 0.1441
Rank 12: Alternative 39, Q = 0.1676, S = 0.1877, R = 0.1441
Rank 13: Alternative 49, Q = 0.1676, S = 0.1877, R = 0.1441
Rank 14: Alternative 72, Q = 0.1764, S = 0.2389, R = 0.1308
Rank 15: Alternative 83, Q = 0.1764, S = 0.2389, R = 0.1308
Rank 16: Alternative 50, Q = 0.1924, S = 0.2313, R = 0.1441
Rank 17: Alternative 58, Q 

Condition Checking Acceptable Advantage (C1) : The difference between the compromise scores of the top two alternatives (Q(A 2 )−Q(A 1 )) must be greater than or equal to DQ, where: DQ= j−1 1​

Here, j is the total number of alternatives. Acceptable Stability (C2) : The best alternative (A 1 ) must also rank first in either the utility measure (S i​) or the regret measure (R i​). If either condition is violated, the compromise solution is extended to include adnal alternatives.

In [202]:
def check_vikor_conditions(Q, S, R, rankings):
    """
    Checks the VIKOR conditions for acceptable advantage and stability.
    Args:
    - Q: The compromise solution (Q_i) for each alternative.
    - S: The utility measure (S_i) for each alternative.
    - R: The regret measure (R_i) for each alternative.
    - rankings: A list of tuples containing the alternative index and its Q, S, and R values, sorted by Q.
    Returns:
    - compromise_solution: The final compromise solution (list of alternatives).
    - conditions_satisfied: Boolean indicating whether both conditions are satisfied.
    """
    j = len(Q)  # Total number of alternatives
    DQ = 1 / (j - 1)  # Threshold for acceptable advantage

    # Extract the top-ranked alternative (A1) and the second-ranked alternative (A2)
    A1_index, A1_values = rankings[0]
    A2_index, A2_values = rankings[1]

    # Condition 1: Acceptable Advantage
    Q_A1 = A1_values[0]  # Q value of A1
    Q_A2 = A2_values[0]  # Q value of A2
    acceptable_advantage = (Q_A2 - Q_A1) >= DQ

    # Condition 2: Acceptable Stability
    # Check if A1 ranks first in either S or R
    S_rankings = sorted(enumerate(S), key=lambda x: x[1])  # Sort by S_i
    R_rankings = sorted(enumerate(R), key=lambda x: x[1])  # Sort by R_i
    A1_S_rank = next(i for i, (index, _) in enumerate(S_rankings) if index == A1_index)
    A1_R_rank = next(i for i, (index, _) in enumerate(R_rankings) if index == A1_index)
    acceptable_stability = (A1_S_rank == 0) or (A1_R_rank == 0)

    # Determine the compromise solution
    compromise_solution = []
    if acceptable_advantage and acceptable_stability:
        # Both conditions are satisfied: A1 is the best alternative
        compromise_solution.append(A1_index)
        conditions_satisfied = True
    else:
        # One or both conditions are violated: Extend the compromise solution
        conditions_satisfied = False
        for rank, (index, (q, _, _)) in enumerate(rankings):
            if q - Q_A1 < DQ:
                compromise_solution.append(index)

    return compromise_solution, conditions_satisfied


# Example Usage
# Assume Q, S, R, and rankings are already calculated
compromise_solution, conditions_satisfied = check_vikor_conditions(Q, S, R, rankings)

# Print results
if conditions_satisfied:
    print("Both conditions are satisfied.")
    print(f"Best Alternative: {compromise_solution[0] + 1}")
else:
    print("One or both conditions are violated.")
    print("Compromise Solution (Extended):", [idx + 1 for idx in compromise_solution])

One or both conditions are violated.
Compromise Solution (Extended): [47, 43]


In [204]:
data['Q'] = Q
data['S'] = S
data['R'] = R
data['Ranking'] = 0
for rank, (index, _) in enumerate(rankings, start=1):
    data.at[index, 'Ranking'] = rank

# Reorder columns as desired
desired_columns = ['Req ID', 'Req Name', 'Description', 'Cost', 'Value', 'Importance', 'Q', 'S', 'R', 'Ranking']
data = data[desired_columns]
data = data.sort_values(by='Ranking', ascending=True)

data.head()

,Req ID,Req Name,Description,Cost,Value,Importance,Q,S,R,Ranking
46,147,Payment Timer,Displays the time left to complete the transac...,3,8,9,0.016227,0.115671,0.072062,1
42,143,Proceed for Payment,To proceed with the payment process,1,7,9,0.023183,0.087217,0.087217,2
89,190,E-Card category,Choose the category of the e-gift card,3,7,9,0.064279,0.159280,0.087217,3
32,133,Book Ticket,To book tickets for a movie,4,9,9,0.067024,0.108094,0.108094,4
35,136,Select Show Time,To select the desired show time,4,8,9,0.091894,0.151702,0.108094,5


In [206]:
output_file = "excel_file/Ranked_Requirements.csv"
data.to_csv(output_file, index=False)

print(f"\nRanked requirements saved to '{output_file}'")


Ranked requirements saved to 'excel_file/Ranked_Requirements.csv'
